In [1]:
import numpy as np
import pandas as pd 
import random
import re
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.optimizers import Adam,RMSprop
from functools import reduce

Using TensorFlow backend.


In [2]:
data = open('dinos.txt', 'r').read()
chars = list(set(data.lower()))
ctoi = { ch:i for i,ch in enumerate(sorted(chars)) }
itoc = { i:ch for i,ch in enumerate(sorted(chars)) }
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))

There are 20969 total characters and 27 unique characters in your data.


In [3]:
with open("dinos.txt") as f:
        examples = f.readlines()

examples = [x.lower().strip() for x in examples]
np.random.seed(0)
np.random.shuffle(examples)

# maxlen is the length of longest word in examples
maxlen = reduce((lambda x, y: max(x,y)), map(lambda x: len(x), examples))

x = np.zeros((len(examples), maxlen, len(chars)), dtype=np.int)
y = np.zeros((len(examples), maxlen, len(chars)), dtype=np.int)
for j in range(len(examples)):
    x[j][0][ctoi["\n"]] = 1
    for k in range(1,maxlen):
        if k <= len(examples[j]):
            x[j][k][ctoi[examples[j][k-1]]] = 1
        else:
            x[j][k][ctoi["\n"]] = 1
       
    for k in range(0,maxlen-2):
        y[j][k] = x[j][k+1]
        
    y[j][maxlen-1][ctoi["\n"]] = 1
    
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [4]:
lstm_out = 200
batch_size = 32

optimizer = Adam(lr=0.01,clipvalue=0.5)
model = Sequential()
model.add(LSTM(lstm_out,  input_shape = (maxlen,vocab_size),recurrent_dropout = 0.2, return_sequences=True))
model.add(Dense(vocab_size,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer=optimizer ,metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 25, 200)           182400    
_________________________________________________________________
dense_1 (Dense)              (None, 25, 27)            5427      
Total params: 187,827
Trainable params: 187,827
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
results = model.fit(x, y, epochs=50, verbose=1)

Epoch 1/50
1623/1623 [==============================] - 2s 1ms/step - loss: 1.5984 - acc: 0.5578
Epoch 2/50
1623/1623 [==============================] - 1s 603us/step - loss: 1.1076 - acc: 0.6804
Epoch 3/50
1623/1623 [==============================] - 1s 620us/step - loss: 0.9534 - acc: 0.7173
Epoch 4/50
1623/1623 [==============================] - 1s 607us/step - loss: 0.8950 - acc: 0.7323
Epoch 5/50
1623/1623 [==============================] - 1s 607us/step - loss: 0.8533 - acc: 0.7453
Epoch 6/50
1623/1623 [==============================] - 1s 609us/step - loss: 0.8242 - acc: 0.7526
Epoch 7/50
1623/1623 [==============================] - 1s 609us/step - loss: 0.7949 - acc: 0.7605
Epoch 8/50
1623/1623 [==============================] - 1s 610us/step - loss: 0.7763 - acc: 0.7645
Epoch 9/50
1623/1623 [==============================] - 1s 610us/step - loss: 0.7555 - acc: 0.7690
Epoch 10/50
1623/1623 [==============================] - 1s 616us/step - loss: 0.7311 - acc: 0.7749
Epoch 11/50

In [100]:
test = np.zeros((maxlen,vocab_size))

for i in range(0,maxlen):
    start_letter = np.random.randint(1, vocab_size)
    test[i][start_letter] = 1
    if i == 0:
        first = itoc[start_letter]
        print(first,end='')


test = np.reshape(test, (1, maxlen, len(chars)))

out = ""
prediction = model.predict(test, verbose=0)
index = sample(prediction[0][0],0.1)
out= out + itoc[index]
for i in range(1,maxlen-1):
    prediction = model.predict(test, verbose=0)
    index = sample(prediction[0][i],0.5)
    out= out + itoc[index]
    test[0][i+1] = prediction[0][i]
    if itoc[index] == "\n":
        print(out)
        break

marosaurus

